In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [32]:
main_page_listing_list = [] # creating list to store search pages of 100 car listings
for i in (range(100)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(i * 100) + "&RPG=100"
    main_page_listing_list.append(url)

In [33]:
print(main_page_listing_list)

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1200&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1300&RPG=100', 'https://www.sgcarmart.com/used_cars/listing.

In [29]:
listing_urls

['https://www.sgcarmart.com/used_cars/info.php?ID=905754&DL=1000',
 'https://www.sgcarmart.com/used_cars/info.php?ID=905866&DL=3454',
 'https://www.sgcarmart.com/used_cars/info.php?ID=905761&DL=2572',
 'https://www.sgcarmart.com/used_cars/info.php?ID=905534&DL=3103']

In [34]:
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []
for page_listing in main_page_listing_list:
    content = requests.get(page_listing)
    soup = BeautifulSoup(content.text, 'lxml')
    links = soup.find_all('a')
    for link in links:
        if 'ID=' in link.get('href') and 'DL=' in link.get('href'):
            listing_url = base_url + link.get('href')
            listing_urls.append(listing_url)
            
    listing_urls = list(set(listing_urls))
    time.sleep(2)

KeyboardInterrupt: 

In [37]:
len(list(set(listing_urls)))

8388

In [173]:
def price_error_handling(data_value):
    # Try-Exception error handling
    
    try:   # First try to deal with values higher than 1000
        price = data_value[1]  # will fail on IndexError if retrieves ['na'] scenario
        price = int(price.split(',')[0] + price.split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
        
    except IndexError:  # Dealing with ['na'] and ['', 900'] scenarios
        try: 
            price = int(data_value[1]) # Will fail on IndexError if ['na'] scenario
        except IndexError:  # Deals with ['na'] scenarios
            price = np.nan  # Stores NA values as nan
    
    return price
def get_brand(url):
    brand = url.find(class_='link_redbanner').text.split()[0]
    return brand

def string_extraction(value):
    try:
        price = int(value.split(',')[0]+value.split(',')[1])
    except:
        try: 
            price = int(value) # Will fail on IndexError if ['na'] scenario
        except:  # Deals with ['na'] scenarios
            price = np.nan
    return price   
        
def get_price(url):
    data = url.find(class_ = 'font_red').text.strip().split('$')
    
    price = price_error_handling(data)
    return price

def get_reg_date(url):
    reg_date = url.find_all(class_='row_bg')[1].find_all('td')[3].text.strip().split('(')[0]
    return reg_date

def get_coe_left(url):
    coe = url.find_all(class_='row_bg')[1].find_all('td')[3].text.strip().split('(')[1].split('COE')[0]
    coe = coe_clean(coe)
    return coe
    
    
def coe_clean(value):
    try:
        idx = value.find('yr')
        if idx != -1:
            year = int(value[idx-1])
    except:
        year = 0
     
    try: 
        idx = value.find('mth')
        if idx != -1:
            month = int(value[idx-1])
    except:
        month = 0
        
       
    try: 
        idx = value.find('days')
        if idx != -1:
            day = int(value[idx-1])
    except:
        day = 0
  
    return year*365 + month * 30 + day

def mileage(url):
    miles = url.find_all(class_='row_info')[0].text.strip()
    miles = miles.split('km')[0]#
    miles = string_extraction(miles)
    return miles

def road_tax(url):
    tax = url.find_all(class_='row_info')[1].text.strip().split(' ')[0].split('$')[1]
#     miles = miles.split('km')[0]#
    tax = string_extraction(tax)
    return tax

def manu_year(url):
    date = url.find_all(class_='row_info')[6].text.strip()
    date = int(date)
#     tax = string_extraction(tax)
    return date

def transmission(url):
    date = url.find_all(class_='row_info')[7].text.strip()
    return date

def depreciation(url):
    dep = url.find_all(class_='row_bg')[1].find_all('td')[1].text.strip().split(' /')[0].split('$')[1]
    dep = string_extraction(dep)
    return dep

def dereg_value(url):
    dep = url.find_all(class_='row_info')[2].text.strip().split(' as')[0].split('$')[1]
    dep = string_extraction(dep)
    return dep

def coe_value(url):
    dep = url.find_all(class_='row_info')[3].text.strip().split('$')[1]
    dep = string_extraction(dep)
    return dep
def engin_capacity(url):
    dep = url.find_all(class_='row_info')[4].text.strip().split(' cc')[0]
    dep = string_extraction(dep)
    return dep

def no_owns(url):
    dep = url.find_all(class_='row_info')[11].text.strip()
    dep = int(dep)
    return dep

def OMV(url):
    dep = url.find_all(class_='row_info')[8].text.strip().split('$')[1]
    dep = string_extraction(dep)
    return dep

def ARF(url):
    dep = url.find_all(class_='row_info')[9].text.strip().split('$')[1]
    dep = string_extraction(dep)
    return dep

def vehicle_type(url):
    types = url.find_all(class_='row_bg1')[0].find('a').text.strip()
    return types

In [174]:
a = requests.get('https://www.sgcarmart.com/used_cars/info.php?ID=883615&DL=1277')
b = BeautifulSoup(a.text, 'lxml')
get_price(b)

58888

In [177]:
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE/KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])
file = 'singapore_car_dateset.csv'

In [ ]:

i = 0 

for listingurl in listing_urls:
    response = requests.get(listingurl)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = get_brand(listing_url)
    df.loc[i, 'PRICE'] = get_price(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = get_reg_date(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manu_year(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = OMV(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = ARF(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_value(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = get_coe_left(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engin_capacity(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = no_owns(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = vehicle_type(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
    df.to_csv(file,index=False)    
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(1)  # Prevents us from getting locked out of the website

In [167]:
df.to_csv(file,index=False)

In [172]:
df['LIS']

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE/KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,NO_OF_OWNERS,VEHICLE_TYPE,MILEAGE_KM
0,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,30800,6270,18-May-2010,NaN,2010.0,742.0,Auto,16141.0,20/06/2020,16716.0,16716.0,16438.0,1467.0,1598.0,3.0,Mid-Sized Sedan,140000.0
1,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,NaN,NaN,07-Oct-2009,NaN,2009.0,NaN,Auto,NaN,20/06/2020,39714.0,1986.0,12916.0,1550.0,2982.0,NaN,Van,NaN
2,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,92500,10590,26-Sep-2017,NaN,2017.0,684.0,Auto,49443.0,20/06/2020,27881.0,31034.0,36001.0,2650.0,1499.0,2.0,MPV,15500.0
3,https://www.sgcarmart.com/used_cars/info.php?I...,Volkswagen,NaN,NaN,30-Nov-2018,NaN,2018.0,976.0,Auto,NaN,20/06/2020,32136.0,36991.0,32000.0,3079.0,1798.0,1.0,Luxury Sedan,NaN
4,https://www.sgcarmart.com/used_cars/info.php?I...,Mercedes-Benz,NaN,NaN,24-Mar-2017,NaN,2017.0,NaN,Manual,NaN,20/06/2020,27595.0,1380.0,46302.0,2463.0,1598.0,NaN,Van,NaN
5,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,318888,NaN,08-Oct-2010,NaN,2009.0,3632.0,Manual,82683.0,20/06/2020,162778.0,162778.0,43334.0,NaN,3797.0,3.0,Sports Car,31888.0
6,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,NaN,NaN,27-Jul-2006,NaN,2006.0,1361.0,Auto,NaN,20/06/2020,19525.0,21478.0,25815.0,401.0,1794.0,2.0,MPV,NaN
7,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,52588,6350,25-Nov-2008,NaN,2008.0,1450.0,Auto,27653.0,20/06/2020,39080.0,39080.0,33377.0,3010.0,1995.0,3.0,Luxury Sedan,103580.0
8,https://www.sgcarmart.com/used_cars/info.php?I...,Daihatsu,26800,5360,18-Mar-2010,NaN,2009.0,566.0,Auto,NaN,20/06/2020,15061.0,15061.0,NaN,NaN,1298.0,3.0,Hatchback,96000.0
9,https://www.sgcarmart.com/used_cars/info.php?I...,Subaru,57800,9330,11-Nov-2015,NaN,2015.0,1208.0,Auto,42558.0,20/06/2020,14948.0,14948.0,58109.0,1946.0,1995.0,1.0,SUV,80644.0
